<a href="https://colab.research.google.com/github/jbuxt/testing/blob/main/GEOM180_Workshop_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GEOM180 Workshop 2: Introduction to GEE**

This workshop will be the first time that you are getting used to using Google Earth Engine. Below are a series of tasks that will enable you to connect to GEE and to begin getting to grips with using it.

We will be working with Landsat 8 Level 2 Surface Reflectence data (https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2).

Work through all of the tasks, try to understand the code that you are using and how these tasks work. You will learn more by playing around with and editing the code itself, rather than just clicking through it all once.

Try to answer the questions as you go through.

By the end of the session you should:


*   Know how to access GEE from Colab
*   Understand how to import raster data from GEE
*   Know the difference between an Image and ImageCollection and the differences in working with them
*   Be able to remove clouds from Landsat image
*   Understand how to inspect data with the `display()` function
*   Know how to add vector data to a map
*   Know how to extra band values to calculate NDVI
*   Have experience with exporting raster data from GEE in order to analyse or plot it in other software (such as QGIS).

Once you have completed these tasks, try to apply some of the code to a different region or time period.

Any questions, just ask Josh, Joseph or Steve, or email Josh on j.buxton@exeter.ac.uk.

Workshop date: 17th November 2023


# **Task 1: Import initial package and validate Google Earth Engine connection.**

These steps are required to connect to Google Earth Engine. You may need to do this everytime you load up Earth Engine!

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

AttributeError: ignored

In [ ]:
import geemap

import os
from geemap.conversion import *

# **Task 2: Connect Google Drive to Colab**

In order to easily access your files which are stored in Google Drive, you should mount Google Drive in colab environment.

In the navigation bar on the left of the screen, click the "Files" button. This will open up a panel which allows you to view all of your files. At the top are four buttons, one of which has the Google Drive logo on it. Click this to mount Google Drive, this will then add a drive folder to the files list.

Alternatively you can do this with the code provided below.

In [ ]:
#Use this code to connect between Google Drive and this colab document.

from google.colab import drive
drive.mount('/content/drive')

# **Task 3: Import initial Landsat 8 image**

Below is code to import a single Landat 8 image which covers the Southwest of England. Run through the code and get to grips with using the geemap interface.

We will be creating a Map object that will appear in your Variable list. We will use this same Map throughout today's workshop, so all of your layers may accumulate on there. If you want to remove these layers and start again, you need to restate your Map variable (with `Map = geemap.Map()`).

Note: Landsat 8 Level-2 images need to be re-scaled to get back to their initial values. For more informaton on why the original values are scaled in the first place, see this [USGS page](https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products).



In [ ]:
#First create our map object and call in our Lnadsat 8 Image
Map = geemap.Map()

L8_image = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_204025_20140709')
# Applies scaling factors.

opticalBands = L8_image.select('SR_B.').multiply(0.0000275).add(-0.2)
thermal_bands = L8_image.select('ST_B.*').multiply(0.00341802).add(149.0)

L8_image = L8_image.addBands(opticalBands,None,True)
L8_image = L8_image.addBands(opticalBands,None,True)

#Here we have set the centre of the map to be the image that we are adding - this can be done for a geometry if preferable
Map.centerObject(L8_image)
#Before adding an image to the map, we need to define its visualisation parameters
visParams = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min':0,'max':0.25};
Map.addLayer(L8_image, visParams, 'TOA');
Map.add('layer_manager')                  #This lets us switch between images
Map

#display(L8_image)

*Use the layer manager to switch layers on and off. This will be useful in later tasks when we're increasing the amount of layers. It is important that you use the command `Map.add('layer_manager')` in order to add this functionality to your maps.*

---
*Test out the pixel inspector tool (top right of the map) to view values from the layers*



---
*Use the display command in the box above to print the metadata for the image. What information can you gain from this?*




---
*How many bands does the image have?*




---
*What does the time property of the image tell you?*




# **Task 4: Remove cloud cover from Landsat image**

We can see that there is a considerable amount of cloud cover in this image. Here we will remove this cloud cover.

Earth Engine has a built in algorithm for creating a "cloud score" for a Landsat image. We will use this to generate the mask.

Landsat 8 has a built in band for pixel quality. This is called `QA_PIXEL`. We can find out more information about the Landsat 8 bands in the [GEE Data Catalog](https://https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands) or by using the `display()` command.

We will use the QA_PIXEL band to mask these clouds.

In [ ]:
#Import the same L8 Image
Map = geemap.Map()

L8_image = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_204025_20140709')

opticalBands = L8_image.select('SR_B.').multiply(0.0000275).add(-0.2)
thermal_bands = L8_image.select('ST_B.*').multiply(0.00341802).add(149.0)

L8_image = L8_image.addBands(opticalBands,None,True)
L8_image = L8_image.addBands(opticalBands,None,True)


#Clouds and their shadows have defined bit values in the QA Pixel band.
#Here we select those values, and then choose the QA band.
#We then mask the pixels that have these cloud values

cloudShadowBitMask = (1 << 3)
cloudsBitMask = (1 << 5)
QA = L8_image.select(['QA_PIXEL'])
cloud_mask = QA.bitwiseAnd(cloudShadowBitMask).eq(0).And(QA.bitwiseAnd(cloudsBitMask).eq(0));

#Note .eq(0) sets these values to 0

#Finally, we update the image mask

L8_cloud_masked = L8_image.updateMask(cloud_mask)

Map.centerObject(L8_image)
visParams = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min':0,'max':0.25};
Map.add('layer_manager')
Map.addLayer(L8_image,visParams,'L8_image')
Map.addLayer(L8_cloud_masked,visParams,'L8_image_masked')
Map





To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=8iAaLQUiE7av7_DHEPhCTi4JPhxZv2yp-zW_mx8S4Fw&tc=ln8blu3tH-P1b0pXK1sPUAKT4KhG2wBLdvx8Fj7w-9s&cc=S9ta-qcqgy4unONobbEegDp0du7yCDz6tiFukgACVs4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlFDBxSazwSe00sF7lM5RVNs0BCWdxwBdonw_bBLggDgOXNLJsOs9k

Successfully saved authorization token.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

*Use the layer manager to toggle the layers on and off. What difference do you notice between the layers?*


---



*What happens when you vary the cloud threshold?*

# **Task 5: Add a vector to the image.**

Here we will add some vector data to this map in order to help focus on a specific geographical region.

For this we will import an existing GEE dataset - the FAO borders dataset. From this, we can select the Cornwall shapefile and add this to the map.

In [ ]:
#We are going to add some vector data - a shapefile of Cornwall

FAO_borders = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

Cornwall = FAO_borders.filter(ee.Filter.eq('ADM2_NAME','Cornwall'))

display(Cornwall)
Map.addLayer(Cornwall,{},"Cornwall")
Map

EEException: ignored

*Inspect the FeatureCollection which contains the shapefile of Cornwall. What data is associated with the region that we have chosen?*


---



*Try to filter by different properties or different values of the same property to see what the output looks like and how this might appear on the map. Note: These extra layers may appear on later maps as we are using the same Map variable*

---



*Don't forget to reset the Cornish shapefile once you're done!*

# **Task 6: Create a cloud-free composite from a Landsat ImageCollection.**

One way to minimise the impact of cloud cover is to create a composite map from images taken at different points in time. This allows us to create a spatially complete image, where the pixels are aggreagated over a time period.

With this method we may gain more spatial clarity, but lose some temporal information.

Below is the code provided to import the Landsat 8 ImageCollect and to filter it both temporally (for 2014) and spatially.

As we are working with an ImageCollection, we have to create a function to map our scaling factor and cloud removal over the entire collection - rather than just applying this to a single image.

We then take the median RGB values across the whole time period - thus creating an (almost) spatially complete image.

In [ ]:
#First we need to define out area of interest - here we have chosen the county of Cornwall

FAO_borders = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

Cornwall = FAO_borders.filter(ee.Filter.eq('ADM2_CODE',40101))

#Then we will import our Landsat ImageCollection
#By filtering the bounds, we will get any Landsat 8 image which intersects with the geometry

L8_dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2014-01-01','2014-12-31').filterBounds(Cornwall)


# Apply scaling factors - here we define a function to map over the whole collection, rather than a single image.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

#Create coud cover remover function to apply to whole ImageCollection
def cloud_masking(image):
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  QA = image.select(['QA_PIXEL'])
  cloud_mask = QA.bitwiseAnd(cloudShadowBitMask).eq(0).And(QA.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(cloud_mask)

L8_dataset = L8_dataset.map(apply_scale_factors)
L8_dataset_cloud_masked = L8_dataset.map(cloud_masking)

L8_dataset_median = L8_dataset_cloud_masked.median()


Map.centerObject(L8_dataset)
visParams = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min':0,'max':0.25};
Map.add('layer_manager')
Map.addLayer(L8_dataset_median,visParams,'L8 2014 Median')
Map.addLayer(Cornwall,{'color':'FF0000'},'Cornwall')
Map

#display(L8_dataset_median)
#display(L8_dataset_cloud_masked)


*Uncomment and run the `display()` command at the bottom of the code block. What differences do you notice between the Landsat 8 imagery before and after you have taken the median of it?*


---



*What do you notice about the areas where data is shown on the map?*


---


*The shapefile has been added to the map and is coloured in red using a HEX colour code. If you are unfamiliar with defining colours this way, look into [HEX color pickers](https://htmlcolors.com/google-color-picker) - Google has a built in one. Try out different hex values to change the colour of the shapefile.*

# **Task 7: Clip this L8 Composite image by the region of interest.**

We often want to focus on a specific region of interest or study site, either during the analysis or when making maps.

Here the region of interest is Cornwall. We will clip our median composite image so that we only retain the area within the Cornish vector.

In [ ]:
#Assuming your variables are still saved from above:


FAO_borders = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

Cornwall = FAO_borders.filter(ee.Filter.eq('ADM2_CODE',40101))

#We use the clip command the only keep the L8 data that is within the boundaries of our feature
L8_dataset_median_clipped = L8_dataset_median.clip(Cornwall)

Map.centerObject(L8_dataset)
visParams = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min':0,'max':0.25};
Map.add('layer_manager')
Map.addLayer(L8_dataset_median,visParams,'L8 2014 Median')
Map.addLayer(L8_dataset_median_clipped,visParams,'L8 2014 Median Clipped')
Map.addLayer(Cornwall,{},'Cornwall')
Map


*Q: There are some pixels that are missing from the map still within the area of interest. What are the different reasons for this happening?*

*Q: What would have happened had we tried to use the `clip` function on the original ImageCollection at the start? How might we have gotten this to work?*

# **Task 8: Calculate NDVI from a single image**

The Normalized difference vegetation index (NDVI) is useful for quantifying vegetation greenness, and can be used a proxy for vegetation health.

NDVI is calculated with the Near-infared band (NIR) and the red band. It is defined as (NIR - red)/(NIR + red).

We will calculate it directly with the bands. It is worth noting that there is a built in function called `normalizedDifference()` which can also be used to calculate it if given the correct bands.



In [ ]:
#We will use the Landsat composite image that we created earlier

median_image = L8_dataset_median_clipped

NIR = median_image.select(['SR_B5'])
RED = median_image.select(['SR_B4'])

NDVI = (NIR.subtract(RED)).divide(NIR.add(RED)).rename("NDVI")
median_image_ndvi = median_image.addBands(NDVI)

#display(median_image_ndvi)

*Uncomment the `display` command to check whether NDVI has been added as a seperate band.*

In [ ]:
#Now we will add this to our map

#We must first define a colour palette. We know that our NDVI values should be between -1 and 1
#Vegetation usually has an NDVI larger than 0.2.
#We define a rather simple colour palette below.

ndviVis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']};

Map.addLayer(median_image_ndvi.select(['NDVI']),ndviVis,'NDVI')
Map

Map(bottom=22420.0, center=[50.3734961443035, -3.8973999023437504], controls=(WidgetControl(options=['position…

*What do you notice about the of built-up areas compared to rural areas? What ways do you think the colour palette could be improved?*




---






*The NDVI value is taken from the annual median image composite. How would this change if we considered the NDVI at different points in the year?*


---
*If we wanted to calculate the NDVI across the whole ImageCollection, rather than for an individual image, we would have to write a function to do so. If you want to test this out, try it now. Otherwise we will do this in next week's workshop.*


**Calculating additional multi-band indices**

While NDVI is one of the most commonly used indices which can be calculated from multi-band data, it is certainly not the only one.

Other vegetation indices exist which are designed to mitigate some of the shortcomings of NDVI, such as its saturation at high levels of vegetation, or are specialised for certain ecoregions.

Other indices are designed to detect built up urban areas or bare soil cover.

For now, we will continue to focus on vegetation indices. The Enhanced Vegetation Index (EVI) is desinged to detect changes in canopy structure and to be more sensitive at higher vegetation levels. It is complimentary to NDVI. EVI is calculated differently for different satellites, but for Landsat 8 the equation is:

$EVI = 2.5\frac{(B5 - B4)}{(B5+6*B4 - 7.5*B2+1)}$

**Note:** The coefficients will be different for different satellites, as will the bands.

Another vegetation index, the Normalized Difference Water Index (NDWI) provides an indication of the water content of leaves. This can be calculated as:

$NDWI = \frac{NIR - SWIR}{NIR + SWIR} $

Where NIR is Near-infared and SWIR is short-wave infared.

**Note:** This NDWI was first proposed in [Gao (1996)](https://www.sciencedirect.com/science/article/abs/pii/S0034425796000673?via%3Dihub). It is worth noting that there is another, different, index called NDWI, also proposed in 1996 by [McFeeters](https://doi.org/10.1080/01431169608948714) which is used to monitor water features. This may be of interest to students who want to look at changes in water body extent or flooding events.

*Take some time to calculate EVI and NDWI with your Landsat 8 composite image. Work out a colour scheme and add this to your map.*

In [ ]:
#Calculate and add EVI Band





#Calculate and add NDWI Band







# **Task 9: Export image from GEE to Google Drive**

As discussed in the seminar, it is possible that you will want to do data pre-processing in GEE and then you may want to work on this data offline in QGIS.

You will now export this Landsat data from GEE to your Google Drive. First, define a new Region of Interest. This should not be too large, i.e. the size of a city is fine, you may struggle if you select the whole of Cornwall. This will tell GEE which area you want the data for.

In [ ]:
#Open an interactive map, and then use the geometry tool to draw a region of interest
#This could be just a simple square, or a more complex polygon.

Map

Map(bottom=11374.0, center=[50.276240127787915, -4.772578997262846], controls=(WidgetControl(options=['positio…

In [ ]:
#Here we save the last drawn geometry as a variable

ROI = Map.draw_last_feature

#Check in the variable list on the left to see if it has appeared
#Alternatively, by displaying it, we can check that it has the coordinates that we would expect it to
display(ROI)

In [ ]:
#Finally, we can export this data to Google Drive
#To do so, we need to define a few things:

geemap.ee_export_image_to_drive(
    median_image_ndvi,                   #This is the image that you want to export to your Google drive
    description='L8_Clipped_ROI',        #This will be the name of the raster file that is exported
    folder='export',                     #This will create a folder in your Google Drive to export the data to
    region=ROI.geometry(),               #As our ROI is a feature, we have to get GEE to use its geometry as the region
    scale=30                             #This is the export resolution in metres - set here to 30 as this is the native Landsat resolution
)

If all goes well, you will find the data in the Google Drive folder that you selected. Try downloading this and importing it into QGIS.

If you have a chosen a region that is too large, one of the following may happen:

1.   The data is exported, but split into multiple raster tiles. These must then be stitched back together offline.
2.   The data is not exported and the computation times out.

One of the ways to solve this issue is to either use a smaller area of interest or to set the export to a lower resolution. It is also possible to force GEE to set a higher limit on the number of pixels before it times out.

If your export does not apear in your Google Drive, it may be worth checking the Task Manager. This does not appear in the Python API, but if you go to the Javascript GEE API, there is a built in task manager with which you can review the status of submitted tasks (even when submitted using the Python API).

We will look more into this next week.




# **Indepent tasks: Repeat the above process, but for a different region or time**

Now that you have worked through this example, hopefully you're feeling more comfortable and may want to try some of this out in a different region.

Consider looking at a different regions or different time frames. Within the UK you could use Landsat to visualise the difference across the landscape in drought years, such as 2022, with those without.

You could also look at areas outside the UK. One example that we discussed in the seminar was Paradise, California, site of the 2018 wildfires.  You will need to find the coordinates of a different region and to filter your data this way. Try to create a composite Landsat 8 image. If you choose this for 2017 and 2019 you will be able to see the effects of the wildfire yourself.

**Next week: Working with the Sentinel satellite**

In the next workshop we'll be working with the higher resolution dataset Sentinel-2. If you finish today's activity, feel free to look into Sentinel-2 and start getting to grips with it.

It is worth remembering that with other datasets the cloud filtering process wil be different. Additionally, the bands will not be the same and may be numbered/ordered differently. Check the GEE Data Catalog for further details on band numbering.

# **Javascript to Python converter**

 `geemap` has a built in function called `geemap.js_snippet_to_py()` which will convert Javascript code to Python. This is particularly useful as a lot of the data catalogue coding is in Javascript.

 Use the code in the GEE data catalog for importing data. Convert this from Javascript to Python with the built in converter.

 Test out other sets of Javascript code to get the hang of it.

 **Note:** The `add_new_cell` variable is designed to automatically generate a new coding block with the Python code in. However, this does not work on Colab (due to security reasons), so it needs to be set to `False` and then there is an additional step that I have provided to print the Python code. This then needs to be copied into a new coding bloc.

In [ ]:
#Paste the Javascript code within the quotation

js_code =  """

"""

In [ ]:
#Here extract the new python code and print it

lines = geemap.js_snippet_to_py(
    js_code, add_new_cell=False, import_ee=False, import_geemap=False, show_map=True
)

for line in lines:
    print(line.rstrip())

*Exercise: Look at the Javascript and Python codes. What differences do you notice? What similarities are there?*








_